In [19]:
# libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist

from curl_cffi import requests as cureq
from bs4 import BeautifulSoup
import time, random

import requests
import os
import warnings
# Hide FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
# creation of dataframe
def get_df(url, columns_remaining):
    # basic headers (impersonate helps but explicit headers are useful)
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
                  " (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/",
    }
    resp = cureq.get(url, headers=headers, impersonate="chrome", timeout=30)

    df = pd.read_html(resp.text)[0]
    df.columns = [' '.join(col).strip() for col in df.columns]
    df = df.reset_index(drop=True)

    # new list w new names
    new_cols = []
    for col in df.columns:
        if 'level_0' in col :
            new_col = col.split()[-1] # extract player's last name
        else:
            new_col = col
        new_cols.append(new_col)

        # renaming of columns
    df.columns = new_cols
    df.fillna(0, inplace=True)

    if 'Playing Time 90s' in new_cols:
            df = df.rename(columns={'Playing Time 90s': '90s'})

    df = df[['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s'] + columns_remaining ]
    df['Age'] = df['Age'].str[:2]
    df['Position'] = df['Pos'].str[:2] # extract primary position
    df['Position_2'] = df['Pos'].str[3:] # extract 2nd position
    df['Nation'] = df['Nation'].str.split(' ').str.get(1)
    df['League'] = df['Comp'].str.split(' ').str.get(1)
    df['League_'] = df['Comp'].str.split(' ').str.get(2)
    df['League'] = df['League'] + ' ' + df['League_']
    df['League'] = df['League'].fillna('Bundesliga')

    df = df.drop(columns=['League_', 'Comp', 'Rk', 'Pos','Born'])

    return df
    

In [21]:
# refining our dataframe
def refine_df(df_combined):
    df_combined =  df_combined.rename(columns={'Per 90 Minutes G-PK': 'Non-penalty goals',
                                                      'SCA SCA': 'Shot creation actions',
                                                      'Aerial Duels Won%': 'Aerial duels won %',
                                                      'Performance Fld': 'Fouls won',
                                                      'Performance Crs': 'Crosses',
                                                      'Performance Recov': 'Ball recoveries',
                                                      'Expected npxG': 'npxG',
                                                      'Expected npxG/Sh': 'npxG/shot',
                                                      'Standard SoT': 'Shots on target',
                                                      'Total Cmp': 'Passes completed',
                                                      'KP': 'Key passes',
                                                      '1/3': 'Final 3rd passes',
                                                      'Long Att': 'Long passes',
                                                      'PrgP': 'Progressive passes',
                                                      'Total Cmp%': 'Pass completion %',
                                                      'Tkl+Int': 'Tackles + interceptions',
                                                      'Blocks Sh': 'Shots blocked',
                                                      'Clr': 'Clearances',
                                                      'Take-Ons Att': 'Dribbles attempted',
                                                      'Take-Ons Succ%': 'Dribbles success %',
                                                      'Carries CPA': 'Carries into penalty area',
                                                      'Receiving Rec': 'Passes received',
                                                      'Receiving PrgR': 'Progressive passes received',
                                                      'Carries PrgC': 'Progressive carries'
                                                      })
    
    numeric_columns = ['Age', '90s', 'Non-penalty goals', 'Shot creation actions', 'Aerial duels won %', 'Fouls won',
                       'Crosses', 'Ball recoveries', 'npxG', 'npxG/shot', 'Shots on target', 'Passes completed',
                       'Key passes', 'Expected xA', 'Final 3rd passes', 'Long passes', 'Progressive passes', 'Pass completion %',
                       'Tackles + interceptions', 'Shots blocked', 'Clearances', 'Dribbles attempted',
                       'Dribbles success %', 'Carries into penalty area', 'Passes received',
                       'Progressive passes received', 'Progressive carries']
    
    for col in numeric_columns:
        df_combined[col] = pd.to_numeric(df_combined[col], errors='coerce')

    per_90_cols = ['Non-penalty goals', 'Shot creation actions', 'Fouls won',
                      'Crosses', 'Ball recoveries', 'npxG', 'Shots on target',
                      'Passes completed', 'Key passes', 'Expected xA', 'Final 3rd passes',
                      'Long passes', 'Progressive passes', 'Tackles + interceptions',
                      'Shots blocked', 'Clearances', 'Dribbles attempted',
                      'Carries into penalty area', 'Passes received',
                      'Progressive passes received', 'Progressive carries']
    
    df_combined = df_combined[df_combined['90s'] > 0]  

    for col in per_90_cols:
        df_combined[col] = (df_combined[col] / df_combined['90s']).round(2)

    return df_combined

In [22]:
def get_df_url():
    url_standard = 'https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats#stats_standard'
    url_shotcreation = 'https://fbref.com/en/comps/Big5/gca/players/Big-5-European-Leagues-Stats#stats_gca'
    url_misc = 'https://fbref.com/en/comps/Big5/misc/players/Big-5-European-Leagues-Stats#stats_misc'
    url_shoot = 'https://fbref.com/en/comps/Big5/shooting/players/Big-5-European-Leagues-Stats#stats_shooting'
    url_passing = 'https://fbref.com/en/comps/Big5/passing/players/Big-5-European-Leagues-Stats#stats_passing'
    url_def = 'https://fbref.com/en/comps/Big5/defense/players/Big-5-European-Leagues-Stats#stats_defense'
    url_poss = 'https://fbref.com/en/comps/Big5/possession/players/Big-5-European-Leagues-Stats#stats_possession'

    df_standard = get_df(url_standard, ['Per 90 Minutes G-PK'])
    df_shotcreation = get_df(url_shotcreation, ['SCA SCA'])
    df_misc = get_df(url_misc, ['Aerial Duels Won%', 'Performance Fld', 'Performance Crs', 'Performance Recov'])
    df_shoot = get_df(url_shoot, ['Expected npxG', 'Expected npxG/Sh', 'Standard SoT'])
    df_passing = get_df(url_passing, ['Total Cmp', 'KP', 'Expected xA', '1/3', 'Long Att', 'PrgP', 'Total Cmp%'])
    df_def = get_df(url_def, ['Tkl+Int', 'Blocks Sh', 'Clr'])
    df_poss = get_df(url_poss, ['Take-Ons Att', 'Take-Ons Succ%', 'Carries CPA', 'Receiving Rec', 'Receiving PrgR',
                                'Carries PrgC'])
    
    dfs = [df_shotcreation, df_misc, df_shoot, df_passing, df_def, df_poss]

    for df in dfs:
        df.drop(columns=['Player','Nation', 'Squad', 'Age', '90s', 'Position', 'Position_2', 'League'], inplace=True)

    # combining dataframes
    frames = [df_standard, df_shotcreation, df_misc, df_shoot, df_passing, df_def, df_poss]
    df = pd.concat(frames, axis=1, join='outer')

    df = refine_df(df)
    df = df[df['Position'] != 'GK'] 
     
    return df 

In [23]:
get_df_url()

,Player,Nation,Squad,Age,90s,Non-penalty goals,Position,Position_2,League,Shot creation actions,...,Pass completion %,Tackles + interceptions,Shots blocked,Clearances,Dribbles attempted,Dribbles success %,Carries into penalty area,Passes received,Progressive passes received,Progressive carries
0,Brenden Aaronson,USA,Leeds United,25.0,7.1,0.02,FW,MF,Premier League,3.10,...,80.4,3.38,0.00,1.41,2.39,47.1,0.28,24.08,5.07,2.39
1,Jones El-Abdellaoui,MAR,Celta Vigo,19.0,1.1,0.00,MF,FW,La Liga,3.64,...,73.3,0.00,0.00,3.64,5.45,16.7,1.82,38.18,20.91,10.91
2,Himad Abdelli,ALG,Angers,25.0,5.9,0.00,MF,,Ligue 1,2.88,...,84.0,3.22,0.51,1.02,3.05,38.9,0.00,48.31,1.69,3.90
3,Ali Abdi,TUN,Nice,31.0,2.8,0.00,DF,MF,Ligue 1,1.43,...,71.4,2.86,0.00,2.14,2.50,42.9,0.36,18.93,3.57,2.14
4,Salis Abdul Samed,GHA,Nice,25.0,3.9,0.00,MF,,Ligue 1,0.51,...,91.2,2.05,0.00,0.51,0.77,33.3,0.26,33.85,0.77,0.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2349,Yanis Zouaoui,ALG,Le Havre,31.0,8.7,0.00,DF,,Ligue 1,2.18,...,70.5,2.18,0.11,2.64,2.53,50.0,0.11,34.83,2.41,2.18
2350,Igor Zubeldia,ESP,Real Sociedad,28.0,11.0,0.00,DF,,La Liga,0.55,...,85.9,2.36,0.45,6.73,0.18,0.0,0.00,41.09,0.00,0.36
2351,Martín Zubimendi,ESP,Arsenal,26.0,9.4,0.02,MF,,Premier League,2.13,...,88.4,3.09,0.11,1.70,0.21,50.0,0.00,53.30,0.85,0.96
2352,Martin Ødegaard,NOR,Arsenal,26.0,2.3,0.00,MF,,Premier League,4.78,...,79.3,1.74,0.00,2.17,2.17,60.0,0.87,50.87,2.17,4.35


In [24]:
df = get_df_url()

In [25]:
# Fill missing in Position_2 with 'None'
df['Position_2'] = df['Position_2'].replace('', 'None')
df.head()

,Player,Nation,Squad,Age,90s,Non-penalty goals,Position,Position_2,League,Shot creation actions,...,Pass completion %,Tackles + interceptions,Shots blocked,Clearances,Dribbles attempted,Dribbles success %,Carries into penalty area,Passes received,Progressive passes received,Progressive carries
0,Brenden Aaronson,USA,Leeds United,25.0,7.1,0.02,FW,MF,Premier League,3.10,...,80.4,3.38,0.00,1.41,2.39,47.1,0.28,24.08,5.07,2.39
1,Jones El-Abdellaoui,MAR,Celta Vigo,19.0,1.1,0.00,MF,FW,La Liga,3.64,...,73.3,0.00,0.00,3.64,5.45,16.7,1.82,38.18,20.91,10.91
2,Himad Abdelli,ALG,Angers,25.0,5.9,0.00,MF,None,Ligue 1,2.88,...,84.0,3.22,0.51,1.02,3.05,38.9,0.00,48.31,1.69,3.90
3,Ali Abdi,TUN,Nice,31.0,2.8,0.00,DF,MF,Ligue 1,1.43,...,71.4,2.86,0.00,2.14,2.50,42.9,0.36,18.93,3.57,2.14
4,Salis Abdul Samed,GHA,Nice,25.0,3.9,0.00,MF,None,Ligue 1,0.51,...,91.2,2.05,0.00,0.51,0.77,33.3,0.26,33.85,0.77,0.51


In [26]:
df['Position_2'].value_counts()

Position_2
None    1533
MF       285
FW       197
DF        76
Name: count, dtype: int64

In [27]:
df.to_csv('combinedplayerstats.csv',index=False)
print("data saved to combinedplayerstats.csv")

data saved to combinedplayerstats.csv
